# Build NCCF.py

In [1]:
import xarray as xr 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from dask.distributed import Client
import os
from scipy import signal
import scipy
from numpy import matlib

from OOI_hydrophone_cloud.processing import NCCF
from OOI_hydrophone_cloud import utils
%load_ext autoreload
%autoreload 2

In [64]:
client = Client()

2022-11-07 22:39:33,505 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-a098k0mv', purging
2022-11-07 22:39:33,505 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yuh203la', purging
2022-11-07 22:39:33,505 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-2vyp93rb', purging
2022-11-07 22:39:33,505 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-h89rrcfn', purging
2022-11-07 22:39:33,506 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-e_7qjjce', purging
2022-11-07 22:39:33,506 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-q8milsbk', purging
2022-11-07 22:39:33,506 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-j4wr9huw', purging

In [65]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33217,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.80 GiB
Comm: tcp://127.0.0.1:36075,Total threads: 4
Dashboard: http://127.0.0.1:43795/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:34377,


In [2]:
b, a = signal.butter(4, [0.01, 0.9], btype='bandpass')
print(b)
print(a)

[ 0.63496904  0.         -2.53987615  0.          3.80981423  0.
 -2.53987615  0.          0.63496904]
[ 1.         -0.73835614 -2.84105805  1.53624064  3.3497155  -1.14722815
 -1.86018017  0.29769033  0.40318603]


## Open Zarr store from cloud

In [3]:
storage_options = {'account_name':'lfhydrophone', 'account_key':os.environ['AZURE_KEY']}
ds = xr.open_zarr('abfs://hydrophonedata/lf_hydrophone_data_test.zarr', storage_options=storage_options)


In [4]:
start_time = pd.Timestamp('2020-07-20')
end_time = pd.Timestamp('2020-07-28')

ds_sliced = utils.slice_ds(ds, start_time, end_time)

In [ ]:
# create template
st, lt = np.meshgrid(np.arange(0, 30, 0.005), np.arange(0, len(ds_sliced.AXCC1)/(30*200)))
st = st.flatten()
lt = lt.flatten()

template = ds_sliced.AXCC1.assign_coords({'st': ('time', st), 'lt': ('time', lt)})
template = template.set_index(time=("st", "lt")).unstack()

In [55]:
pp = NCCF.preprocess(ds_sliced.AXCC1[:72000])

In [58]:
pp

<xarray.DataArray (time: 72000)>
array([-0.00529565, -0.00766747, -0.00323617, ...,  0.0053828 ,
       -0.00207635,  0.0036139 ])
Dimensions without coordinates: time

In [63]:
cc_pp = ds_sliced.AXCC1.map_blocks(NCCF.preprocess, template = ds_sliced.AXCC1)
ec_pp = ds_sliced.AXEC2.map_blocks(NCCF.preprocess, template = ds_sliced.AXEC2)

In [66]:
cc_pp = cc_pp.compute()
ec_pp = ec_pp.compute()

/home/jhrag/anaconda3/envs/ooi_dev/lib/python3.10/site-packages/distributed/worker.py:2936: UserWarning: Large object of size 5.49 MiB detected in task graph: 
  ([('time',), <xarray.IndexVariable 'time' (time: 7 ... 64[ns]'), {}],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


In [70]:
W = 30
Fs = 200

cc_pp_rs = np.reshape(cc_pp.values, (int(len(cc_pp)/(W*Fs)), int(W*Fs)))
ec_pp_rs = np.reshape(ec_pp.values, (int(len(ec_pp)/(W*Fs)), int(W*Fs)))

In [83]:
R = np.mean(signal.fftconvolve(cc_pp_rs, np.flip(ec_pp_rs, axis=1), axes=1, mode='full'), axis=0)

: 

: 

In [ ]:
plt.plot(R)